In [1]:
example_name = "500_Movie"
example_dir = f"example_dirs/{example_name}"
import os
repo_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
if not os.path.basename(os.getcwd()) == example_name:
    !mkdir -p $example_dir
    os.chdir(example_dir)
print(f"Current working directory: {os.getcwd()}")
print(f"Repository root directory: {repo_root}")

Current working directory: /workspaces/fargocpt/example_dirs/500_Movie
Repository root directory: /workspaces


In [2]:
configfile = "setup.yml"
!cp $repo_root/fargocpt/examples/config.yml $configfile

In [3]:
import ruamel.yaml
yaml = ruamel.yaml.YAML()
with open(configfile, "r") as infile:
    config = yaml.load(infile)

In [4]:
config["nbody"][1]["accretion efficiency"] = "2"
config["MonitorTimestep"] = 0.0314 # monitor scalar files around every half orbit
config["Nmonitor"] = 5 # write a snapshot after every 20 monitor timesteps = every orbit
config["Nsnapshots"] = 500
config["cps"] = 4 # use medium resolution, this is only for show, but it should still look nice
config["Frame"] = "F"
config["OmegaFrame"] = 0
config["nbody"][1]["ramp-up time"] = 10
config["nbody"][1]["accretion efficiency"] = 10
config["DiskFeedback"] = "yes"


with open(configfile, "w") as outfile:
    yaml.dump(config, outfile)

In [5]:
from fargocpt import run
np = 2 # Number of mpi processes. Should be equal to the number of numa nodes on your machine, check your cluster docu or run `lscpu` or `./run_fargo --print-numa` if you're on linux.
nt = 1 # Number of threads per mpi process, set it to the number of cores you want to use / number of MPI processes
run(["start", configfile], np=np, nt=nt, exe=repo_root+"/fargocpt/bin/fargocpt_exe", detach=False)

Running command: mpirun -np 2 --report-pid /tmp/tmptwamgvu_ -x OMP_NUM_THREADS=1 /workspaces/fargocpt/bin/fargocpt_exe start setup.yml
fargo process pid 74073

[1] MPI rank #  1 runs as process 74084
[0] MPI rank #  0 runs as process 74083
[0] MPI rank #  0 OpenMP thread #  0 of  1 on codespaces-f4c61d
[1] MPI rank #  1 OpenMP thread #  0 of  1 on codespaces-f4c61d
[0] fargo: This file was compiled on May 15 2024, 21:56:12.
[0] fargo: Last git commit: e9b10d840e009719d3266c72c13697b63ce658e2
[0] fargo: Files changed since git commit: 
[0] fargo: This version of FARGO used _GNU_SOURCE
[0] fargo: This version of FARGO used NDEBUG. So no assertion checks!
[0] Using parameter file setup.yml
[0] Computing disk quantities within 5.00000e+00 L0 from coordinate center
[0] BC: Inner composite = reflecting
[0] BC: Outer composite = reflecting
[0] BC: Sigma inner = zerogradient
[0] BC: Sigma outer = zerogradient
[0] BC: Energy inner = zerogradient
[0] BC: Energy outer = zerogradient
[0] BC: Vrad 

0

In [6]:
import numpy as np
import matplotlib.colors as mplcolors
import matplotlib.pyplot as plt

def plot_field(loader, name, N, ax=None, dataunit=None, vmin=None, vmax=None, cmap="viridis", title=None):
    R, PHI, vals = loader.gas.vars2D.get(name, N, grid_for_plot=True)
    if dataunit is None:
        dataunit = vals.unit
    C = vals.to_value(dataunit)

    X = R*np.cos(PHI)
    Y = R*np.sin(PHI)

    if ax is None:
        fig, ax = plt.subplots(dpi=150)
    else:
        fig = ax.get_figure()

    norm = mplcolors.Normalize(vmin=vmin, vmax=vmax)

    # Hacky way to support arrays that are defined on the radial interfaces
    if C.shape[0] == X.shape[0]:
        C = C[:-1,:]

    pcm = ax.pcolormesh(X,Y,C, norm=norm, cmap=cmap)
    ax.set_aspect("equal")

    t = loader.snapshot_time[N].to_value("kyr")
    if title is None:
        title = ""
    else:
        title += "\n"
    title += f" t={t:.2e}kyr, N={N}"
    ax.set_title(title)

    cbar = fig.colorbar(pcm, ax=ax)
    cbar.set_label(f"{name} [{dataunit}]")
    
    return fig

In [7]:
from fargocpt import Loader
l = Loader("output/out/")

In [8]:
folder = "imgs_1"
!mkdir -p $folder

In [9]:
from tqdm import tqdm
for n in tqdm(l.snapshots):
    fig = plot_field(l, "Sigma", n, dataunit="g/cm2", vmin=0, vmax=600, cmap="magma", title="Sigma");
    fig.savefig(f"imgs/Sigma{n}.jpg", dpi=300)
    plt.close(fig)

100%|██████████| 501/501 [01:32<00:00,  5.43it/s]


In [12]:
import os
import moviepy.video.io.ImageSequenceClip
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_files = []

for img_number in range(1,567): 
    image_files.append('imgs/Sigma' + str(img_number) + '.jpg') 

fps = 30

clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('500_Movie_1.mp4')

Moviepy - Building video 500_Movie_1.mp4.
Moviepy - Writing video 500_Movie_1.mp4



t:  63%|██████▎   | 359/567 [00:10<00:06, 32.21it/s, now=None]kill: (74073): No such process


Moviepy - Done !
Moviepy - video ready 500_Movie_1.mp4
